In [111]:
import pandas as pd
import numpy as np
from fpdf import FPDF
import matplotlib.pyplot as plt
import os
from datetime import datetime
from datetime import date

def load_data():
    df_xg = pd.read_csv('C:/Users/SéamusPeareBartholdy/Documents/GitHub/AC-Horsens-U19/xg.csv')
    
    df_possession_stats = pd.read_csv('C:/Users/SéamusPeareBartholdy/Documents/GitHub/AC-Horsens-U19/terr_poss.csv')
    
    df_xg_agg = pd.read_csv('C:/Users/SéamusPeareBartholdy/Documents/GitHub/AC-Horsens-U19/xg_agg.csv')
    
    penalty_area_entry_counts = pd.read_csv('C:/Users/SéamusPeareBartholdy/Documents/GitHub/AC-Horsens-U19/penalty_area_entry_counts.csv')
    
    df_matchstats = pd.read_csv('C:/Users/SéamusPeareBartholdy/Documents/GitHub/AC-Horsens-U19/matchstats.csv')
    
    df_ppda = pd.read_csv('C:/Users/SéamusPeareBartholdy/Documents/GitHub/AC-Horsens-U19/ppda.csv')
    
    return df_xg, df_possession_stats,df_xg_agg, penalty_area_entry_counts, df_matchstats, df_ppda




In [116]:
df_grouped = df_possession_stats.groupby(['team.name', 'label', 'territorial_possession']).size().reset_index(name='count')
df_total_possessions = df_grouped.groupby(['label'])['count'].sum().reset_index()
df_total_possessions = df_total_possessions.rename(columns={'count': 'total_possessions'})
df_total_possessions = df_grouped.merge(df_total_possessions)
df_total_possessions['percentage_possession'] = (df_total_possessions['count'] / df_total_possessions['total_possessions']) * 100
df_total_possessions = df_total_possessions.groupby(['team.name','label'])['percentage_possession'].mean().reset_index()
df_total_possessions = df_total_possessions[['team.name','label','percentage_possession']]
print(df_total_possessions)

     team.name                                    label  percentage_possession
0      AGF U19                 AGF U19 - AaB U19, 2 - 1              15.599922
1      AGF U19         AGF U19 - Brøndby U19, 2 - 2 (P)              17.611143
2      AGF U19             AGF U19 - Esbjerg U19, 0 - 4              20.077614
3      AGF U19             AGF U19 - Horsens U19, 1 - 2              17.444379
4      AGF U19           AGF U19 - København U19, 1 - 3              14.380265
..         ...                                      ...                    ...
335  Vejle U19  Vejle U19 - Nordsjælland U19, 1 - 1 (E)              14.155496
336  Vejle U19                Vejle U19 - OB U19, 0 - 2              15.933859
337  Vejle U19           Vejle U19 - Randers U19, 0 - 1              17.001630
338  Vejle U19         Vejle U19 - Silkeborg U19, 0 - 3              17.869034
339  Vejle U19       Vejle U19 - SønderjyskE U19, 1 - 4              17.296804

[340 rows x 3 columns]
